In [1]:
# 导入依赖包
import os
import csv
import numpy as np
import pandas as pd
import json
from nltk import word_tokenize
from nltk.corpus import words
import nltk
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
import copy
# 导入自定义依赖包
import LoadFileDemo as lf
import SeparateWordDemo as sw
import WordCleanDemo as wc

In [42]:
# 输入多篇文章样本（不分词）进行短语权重分析
def tfidf_tlist(text_list = ['']):
    #该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频 
    vectorizer=CountVectorizer()
    #该类会统计每个词语的tf-idf权值  
    transformer=TfidfTransformer()
    #第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
    tfidf=transformer.fit_transform(vectorizer.fit_transform(text_list))
    #获取词袋模型中的所有词语  
    word=vectorizer.get_feature_names()
    #将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重  
    weight=tfidf.toarray()
    return word,weight

# 输入多篇文章样本（分词后）进行短语权重分析
def tfidf_wlist(text_list = [['']]):
    text_list_org = [" ".join(text) for text in text_list]
    return tfidf_tlist(text_list_org)

# 数组元素调换位置
def change_pos(arr,first_idx,second_idx):
    if first_idx>=0 and second_idx>0 and first_idx<=len(arr) and second_idx<=len(arr):
        temp_value = arr[second_idx]
        arr[second_idx] = arr[first_idx]
        arr[first_idx] = temp_value

# 找一个样本的所有单词权重最top的k个数组下标
def find_topk_tfidf_single(weight_row = [],topk = 10):
    if len(weight_row) < topk : topk=len(weight_row)
    weight_row_temp = copy.deepcopy(weight_row)
    org_idx = [i for i in range(len(weight_row_temp))]
    for i in range(topk):
        max_idx = i
        for j in range(len(weight_row_temp)-i):
            #找出最第i位top的置换到weight_row_temp的i位置
            if weight_row_temp[max_idx] < weight_row_temp[j+i] : max_idx = j+i
        change_pos(weight_row_temp,i,max_idx)
        change_pos(org_idx,i,max_idx)
    return org_idx[:topk]

def find_topk_tfidf(weight = [],topk = 10):
    return [find_topk_tfidf_single(weight[j],topk) for j in range(len(weight))]


In [43]:
lf.log('测试短语加权分析')
corpus=["我 来到 北京 清华大学",
        "他 来到 了 网易 杭研 大厦",
        "小明 硕士 毕业 与 中国 科学院",
        "我 爱 北京 天安门"]

word,weight=tfidf_tlist(corpus)
    
#遍历各个样本的所有单词的权重向量（第一个for）
#遍历每个样本对应每个单词的权重（第二个for）
for i in range(len(weight)):  
    print("这里输出第",i,u"类文本的词语tf-idf权重：")
    for j in range(len(word)):  
        print(word[j],weight[i][j])

lf.log('测试短语加权排序')
#定义一个数组topk_tfidf，记录weight最top的k个单词下标
topk = 10
topk_tfidf = find_topk_tfidf(weight,topk)
print('权重排序：%s' % topk_tfidf)


------------------测试短语加权分析-------------------
这里输出第 0 类文本的词语tf-idf权重：
中国 0.0
北京 0.52640543361
大厦 0.0
天安门 0.0
小明 0.0
来到 0.52640543361
杭研 0.0
毕业 0.0
清华大学 0.66767854461
硕士 0.0
科学院 0.0
网易 0.0
这里输出第 1 类文本的词语tf-idf权重：
中国 0.0
北京 0.0
大厦 0.525472749264
天安门 0.0
小明 0.0
来到 0.414288751166
杭研 0.525472749264
毕业 0.0
清华大学 0.0
硕士 0.0
科学院 0.0
网易 0.525472749264
这里输出第 2 类文本的词语tf-idf权重：
中国 0.4472135955
北京 0.0
大厦 0.0
天安门 0.0
小明 0.4472135955
来到 0.0
杭研 0.0
毕业 0.4472135955
清华大学 0.0
硕士 0.4472135955
科学院 0.4472135955
网易 0.0
这里输出第 3 类文本的词语tf-idf权重：
中国 0.0
北京 0.61913029649
大厦 0.0
天安门 0.78528827571
小明 0.0
来到 0.0
杭研 0.0
毕业 0.0
清华大学 0.0
硕士 0.0
科学院 0.0
网易 0.0
------------------测试短语加权排序-------------------
权重排序：[[8, 1, 5, 3, 4, 2, 6, 7, 0, 9], [2, 6, 11, 5, 4, 3, 1, 7, 8, 9], [0, 4, 7, 9, 10, 5, 6, 2, 8, 3], [3, 1, 2, 0, 4, 5, 6, 7, 8, 9]]


In [44]:
lf.log('测试加载样本数据')
path_folder = '/Users/hata/Develop/' \
    + 'workspace_springmvc/' \
    + 'SpringbootRESTfulDemo/'
data_folder=os.path.join(path_folder,"src")
print('导入的数据集是： %s ' % data_folder)
file_type = ['txt','java','xml']
documents, filenames, paths = lf.load_project_data(data_folder)
lf.log('检查加载样本数据')
print('documents length: %i ' % len(documents))
print('filenames length: %i ' % len(filenames))
print('paths length: %i ' % len(paths))


------------------测试加载样本数据-------------------
导入的数据集是： /Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src 
输入目录地址：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src
－－－－－－－－－－－
当前目录地址：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src
含有文件：['.DS_Store']
读取文件错误 : /Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/.DS_Store 
－－－－－－－－－－－
当前目录地址：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main
含有文件：['.DS_Store']
读取文件错误 : /Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/.DS_Store 
－－－－－－－－－－－
当前目录地址：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/java
含有文件：['.DS_Store']
读取文件错误 : /Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/java/.DS_Store 
－－－－－－－－－－－
当前目录地址：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/java/demo
含有文件：['.DS_Store']
读取文件错误 : /Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/java/d

In [45]:
lf.log('测试分词效果')
output_text_list = sw.separate_texts(documents)
print('output_text_list length: %i ' % len(output_text_list))


------------------测试分词效果-------------------
分词符号集合: ['\r\n', '\r', '\n', '\t', ' ', '_', '.', ',', '+', '-', '=', ';', ':', '/', '*', '#', '@', '?', '!', '%', '^', "'", '"', '(', ')', '[', ']', '{', '}', '&', '~', '|', '-'] 
目标分词符号:   
分词符号集合: ['\r\n', '\r', '\n', '\t', ' ', '_', '.', ',', '+', '-', '=', ';', ':', '/', '*', '#', '@', '?', '!', '%', '^', "'", '"', '(', ')', '[', ']', '{', '}', '&', '~', '|', '-'] 
目标分词符号:   
分词符号集合: ['\r\n', '\r', '\n', '\t', ' ', '_', '.', ',', '+', '-', '=', ';', ':', '/', '*', '#', '@', '?', '!', '%', '^', "'", '"', '(', ')', '[', ']', '{', '}', '&', '~', '|', '-'] 
目标分词符号:   
分词符号集合: ['\r\n', '\r', '\n', '\t', ' ', '_', '.', ',', '+', '-', '=', ';', ':', '/', '*', '#', '@', '?', '!', '%', '^', "'", '"', '(', ')', '[', ']', '{', '}', '&', '~', '|', '-'] 
目标分词符号:   
分词符号集合: ['\r\n', '\r', '\n', '\t', ' ', '_', '.', ',', '+', '-', '=', ';', ':', '/', '*', '#', '@', '?', '!', '%', '^', "'", '"', '(', ')', '[', ']', '{', '}', '&', '~', '|', '-'] 
目标分词符号:

In [46]:
lf.log('测试特征预处理（全部）')
text_done_list = wc.word_clean(output_text_list)
print('text_done_list length: %i ' % len(text_done_list))
#print(text_done_list[0])


------------------测试特征预处理（全部）-------------------
本次处理文章的长度：20 
本次处理文章的长度：38 
本次处理文章的长度：246 
本次处理文章的长度：300 
本次处理文章的长度：347 
本次处理文章的长度：35 
本次处理文章的长度：802 
本次处理文章的长度：2896 
本次处理文章的长度：307 
本次处理文章的长度：129 
本次处理文章的长度：54 
本次处理文章的长度：77 
本次处理文章的长度：382 
本次处理文章的长度：47 
本次处理文章的长度：88 
本次处理文章的长度：494 
本次处理文章的长度：1849 
text_done_list length: 17 


In [47]:
lf.log('测试短语加权和权重排序')
topk_t = 10
word_t,weight_t=tfidf_wlist(text_done_list)
topk_tfidf_t = find_topk_tfidf(weight_t,topk_t)
print('topk_t : %i ' % topk_t)
print('word_t length: %i ' % len(word_t))
print('weight_t length: %i ' % len(weight_t))
print('weight_t[0] length: %i ' % len(weight_t[0]))

lf.log('显示前20个文本样例')
for i in range(len(filenames[:20])):
    print('第 %s 个文件' % str(i+1))
    print('文件名称：%s' % filenames[i])
    print('文件路径：%s' % paths[i])
    print('topk排序：%s' % topk_tfidf_t[i])
    print('词语：%s' % [word_t[index] for index in topk_tfidf_t[i]])
    print('权重：%s' % [weight_t[i][index] for index in topk_tfidf_t[i]])


------------------测试短语加权和权重排序-------------------
topk_t : 10 
word_t length: 289 
weight_t length: 17 
weight_t[0] length: 289 
------------------显示前20个文本样例-------------------
第 1 个文件
文件名称：App.java
文件路径：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/java/demo/java/spring
topk排序：[112, 283, 11, 145, 13, 178, 189, 239, 231, 74]
词语：['hello', 'world', 'app', 'main', 'args', 'out', 'println', 'system', 'string', 'demo']
权重：[0.55817164181001544, 0.55817164181001544, 0.27908582090500772, 0.2436927605154294, 0.21858098453353969, 0.21858098453353969, 0.21858098453353969, 0.21858098453353969, 0.14779486375438311, 0.097571311790603646]
第 2 个文件
文件名称：SpringdemoApplication.java
文件路径：/Users/hata/Develop/workspace_springmvc/SpringbootRESTfulDemo/src/main/java/demo/java/spring
topk排序：[12, 30, 226, 13, 225, 227, 19, 207, 177, 145]
词语：['application', 'boot', 'springdemo', 'args', 'spring', 'springframework', 'autoconfigure', 'run', 'org', 'main']
权重：[0.7335835402040719, 0.489055693

In [48]:
#print(word_t[:300])
#print(weight_t[3][:300])